In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 4 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

14 November 2022 ; 21:09:23


In [3]:
features = ['VORP','PTS','WS','OWS','FG','MP','OBPM','PER','BPM','Seed']
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/smoy/smoy_data.csv')

# Load to-be-predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('smoy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,Precious Achiuwa,C,23,TOR,12,0,20.4,3.0,7.7,0.391,...,2.36,114.37,112.28,2.09,3.24,114.9,111.9,3.0,0,1
2,Dalano Banton,PG,23,TOR,13,0,9.5,1.8,4.3,0.429,...,2.36,114.37,112.28,2.09,3.24,114.9,111.9,3.0,0,1
4,Khem Birch,C,30,TOR,4,0,8.5,0.8,1.3,0.600,...,2.36,114.37,112.28,2.09,3.24,114.9,111.9,3.0,0,1
5,Chris Boucher,PF,30,TOR,11,0,21.5,4.1,8.2,0.500,...,2.36,114.37,112.28,2.09,3.24,114.9,111.9,3.0,0,1
6,Justin Champagnie,SF,21,TOR,2,0,4.5,1.5,1.5,1.000,...,2.36,114.37,112.28,2.09,3.24,114.9,111.9,3.0,0,1


In [5]:
# Set training data
train_data = train_data[train_data['Year'] < 2020] # DELETE THIS LINE
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP', 'G', 'GS', 'Tm', 'PER', 'VORP', 'WS', 'PTS', 'Seed', 'OBPM', 'OWS', 'Share']]
res.head(5)

,Player,MP,G,GS,Tm,PER,VORP,WS,PTS,Seed,OBPM,OWS,Share
0,Bobby Portis,26.8,12,3,MIL,18.3,2.050000,7.516667,13.3,1,1.6,2.733333,0.228908
1,Kevin Love,21.2,13,1,CLE,20.6,3.784615,7.569231,12.2,3,5.7,5.046154,0.201098
2,Bones Hyland,20.8,9,0,DEN,15.6,0.911111,1.822222,14.0,1,1.6,0.911111,0.135245
3,Collin Sexton,22.1,15,1,UTA,16.3,0.546667,4.373333,14.0,3,0.0,2.733333,0.117995
4,Malcolm Brogdon,23.0,11,0,BOS,18.6,1.490909,4.472727,13.7,2,2.0,2.981818,0.112040


In [7]:
res.to_csv(path + f'Results/smoy/results_week_{week}.csv', index = None)